In [5]:
import sys
import os
sys.path.append(os.path.abspath('../src'))

from dash import Dash, html, dcc, callback, Output, Input
import pandas as pd
import plotly.express as px
from prophet.plot import plot_plotly
from datetime import datetime, timedelta
from plot import plot_data
from prediktiv_analyse import prediktiv_analyse
from statistikk import statistikk
from hent_lagre import hent 
import dash_leaflet as dl
import os

csv_path = r'../data/lokasjonsdata.csv'

#laster inn nåværende csv fil, ellers oppretter ny
if os.path.exists(csv_path) and os.path.getsize(csv_path) > 0:
    df = pd.read_csv(csv_path)

else:
    df = pd.DataFrame(columns=['Date'])
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)
    df.to_csv(csv_path, index=True) 

#init dash app
app = Dash()

#html-dritt
app.layout = html.Div(style={'fontFamily': 'Arial, sans-serif', 'padding': '20px'}, children=[
    html.H1("Heftig interaktiv visualisering", style={'textAlign': 'center', 'fontSize': '36px'}),

    html.Div(style={'display': 'flex', 'flexWrap': 'wrap', 'gap': '20px', 'marginBottom': '30px'}, children=[
        html.Div([
            html.Label("start-dato for data:", style={'fontSize': '18px'}),
            dcc.DatePickerSingle(id='Start_dato-velger', date=datetime.today() - timedelta(days=5*365), style={'fontSize': '16px'})
        ]),
        html.Div([
            html.Label("slutt-dato for data:", style={'fontSize': '18px'}),
            dcc.DatePickerSingle(id='Slutt-dato-velger', date=datetime.today() - timedelta(days=31), style={'fontSize': '16px'})
        ]),
        html.Div([
            html.Label("Plot-type:", style={'fontSize': '18px'}),
            dcc.Dropdown(
                options=[
                    {'label': 'Visualisering', 'value': 'visualisering'},
                    {'label': 'Statistikk', 'value': 'statistikk'},
                    {'label': 'Prediktiv Analyse', 'value': 'prediktiv_analyse'}
                ],
                value='visualisering',
                id='Plot-velger',
                style={'width': '220px', 'fontSize': '16px'})
        ]),
        html.Div([
            html.Label("Velg By:", style={'fontSize': '18px'}),
            dcc.Dropdown(
            options=[],
            value=None,
            id='By-velger',
            style={'width': '220px', 'fontSize': '16px'}
            )
        ]),
        html.Div([
            html.Label("Juster verdi:", style={'fontSize': '18px'}),
            dcc.Slider(
                min=1, max=10, step=1, value=1,
                id='slider',
                marks={i: str(i) for i in range(1, 11)},
                tooltip={"placement": "bottom", "always_visible": True},
            )
        ])
    ]),

    html.Div(style={'display': 'flex', 'gap': '20px', 'marginBottom': '40px'}, children=[
        html.Div([
            html.H2("Koordinat-velger", style={'textAlign': 'center', 'fontSize': '24px'}),
            dl.Map(center=[63.4305, 10.3951], zoom=10, children=[
                dl.TileLayer(),
                dl.LayerGroup(id="marker-layer")
            ], id="map", style={'width': '100%', 'height': '500px'}),
            html.Div(id="click-coordinates", style={'fontSize': '16px', 'textAlign': 'center', 'marginTop': '10px'})
        ], style={'flex': '1'}),

        html.Div([
            html.H2("Data-visualisering", style={'textAlign': 'center', 'fontSize': '24px'}),
            dcc.Graph(id='graph-content', style={'height': '500px'})
        ], style={'flex': '1'})
    ])
])

#callback
@callback(
    Output('graph-content', 'figure'),
    Output('click-coordinates', 'children'),
    Output('marker-layer', 'children'),
    Output('By-velger', 'options'),
    Output('By-velger', 'value'),   
    Input('Plot-velger', 'value'),
    Input('By-velger', 'value'),
    Input('slider', 'value'),
    Input("map", "clickData"),
    Input('Start_dato-velger', 'date'),
    Input('Slutt-dato-velger', 'date')
)   
def update_graph(plot, by, slider, click_data, start_dato, slutt_dato):
    df = pd.read_csv(csv_path)
    startdato = int(datetime.fromisoformat(start_dato).strftime("%Y%m%d"))
    sluttdato = int(datetime.fromisoformat(slutt_dato).strftime("%Y%m%d"))
    
    if click_data and "latlng" in click_data:
        lat = click_data["latlng"]["lat"]
        lon = click_data["latlng"]["lng"]
        print(f"Henter data for lat={lat}, lon={lon}, start={startdato}, slutt={sluttdato}")
        hent(lat, lon, startdato, sluttdato, df)
        df = pd.read_csv(csv_path)

    city_columns = [col for col in df.columns if col != 'Date']
    options = [{'label': col, 'value': col} for col in city_columns]
    value = by if by in city_columns else (city_columns[0] if city_columns else None)

    #velg fremstilling
    if value is not None:
        if plot == 'visualisering':
            fig = plot_data(value, df)
        elif plot == 'statistikk':
            fig = statistikk(value, df)
        elif plot == 'prediktiv_analyse':
            modell, forecast = prediktiv_analyse(slider, value, df[['Date', value]])
            fig = plot_plotly(modell, forecast)
        else:
            fig = {}
    else:
        fig = {}

    if click_data and "latlng" in click_data:
        lat = click_data["latlng"]["lat"]
        lon = click_data["latlng"]["lng"]
        coords_text = f"Latitude: {lat:.4f}, Longitude: {lon:.4f}"
        marker = dl.Marker(position=[lat, lon], children=dl.Tooltip(f"{lat:.4f}, {lon:.4f}"))
    else:
        coords_text = "Klikk på kartet for koordinater"
        marker = None

    return fig, coords_text, [marker] if marker else [], options, value

In [ ]:
#kjør denne for å kjøre appen i browseren
app.run(jupyter_mode="external",port=8050)

Dash app running on http://127.0.0.1:8050/


Henter data for lat=63.43004121468043, lon=10.401306152343752, start=20200521, slutt=20250419
Response status code: 200
Response URL: https://power.larc.nasa.gov/api/temporal/daily/point?parameters=ALLSKY_SFC_SW_DWN&community=SB&longitude=10.401306152343752&latitude=63.43004121468043&start=20200521&end=20250419&format=JSON
Data lagret i ../data/lokasjonsdata.csv.
Henter data for lat=59.909507176556964, lon=10.700683593750002, start=20200521, slutt=20250419
Response status code: 200
Response URL: https://power.larc.nasa.gov/api/temporal/daily/point?parameters=ALLSKY_SFC_SW_DWN&community=SB&longitude=10.700683593750002&latitude=59.909507176556964&start=20200521&end=20250419&format=JSON
Data lagret i ../data/lokasjonsdata.csv.
Henter data for lat=59.909507176556964, lon=10.700683593750002, start=20200521, slutt=20250419
Response status code: 200
Response URL: https://power.larc.nasa.gov/api/temporal/daily/point?parameters=ALLSKY_SFC_SW_DWN&community=SB&longitude=10.700683593750002&latitud

In [ ]:
#kjør denne for å kjøre appen i jupyter, 
#ikke brukt mye når jeg lagde koden, kan hende den ikke funker
app.run(jupyter_mode="inline",port=8050)